In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('')

Arrests in the UK by ethnicity by area across uk (city(urban area), small town/village(rural b area)) 

First use a classification method to classify what sort of area places are

get arrest data on a mix of different places from each category from different regions in the uk over a set period




How can you remove ai bias from an algorithm which is designed to find social bias?
